<a href="https://colab.research.google.com/github/brlvr/prompt-engineering-workshop/blob/main/04_Separating_Data_and_Instructions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 4: Separating Data and Instructions

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [ ]:
!git clone https://github.com/brlvr/prompt-engineering-workshop/

In [ ]:
%cd prompt-engineering-workshop

In [ ]:
# Import python's built-in regular expression library
import re
import json

# Import the hints module from the utils package
from utils import hints

In [ ]:
import warnings
from huggingface_hub import InferenceClient

# Suppress FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)

from google.colab import userdata
API_KEY = userdata.get('API_KEY')

def get_completion(prompt, system_prompt=None, api_key=API_KEY, stream=False, temperature=0.0, max_tokens=300):
    # Initialize the Hugging Face inference client with the provided API key
    client = InferenceClient(api_key=api_key)

    # Setting up the message for the model input
    messages = [{"role": "user", "content": prompt}]

    # Add the system prompt if provided
    if system_prompt:
        messages.insert(0, {"role": "system", "content": system_prompt})

    try:
        if stream:
            # Streaming the response from the Hugging Face API
            for message in client.chat_completion(
                model="meta-llama/Llama-3.2-1B-Instruct",  # Update with the desired model ID
                messages=messages,
                max_tokens=max_tokens,  # Add max_tokens parameter
                temperature=temperature,  # Add temperature parameter
                stream=True
            ):
                # Printing the streamed response tokens as they arrive
                print(message.choices[0].delta.content, end="")
        else:
            # Non-streaming call, returning the full response at once
            response = client.chat_completion(
                model="meta-llama/Llama-3.2-1B-Instruct",  # Update with the desired model ID
                messages=messages,
                max_tokens=max_tokens,  # Add max_tokens parameter
                temperature=temperature,  # Add temperature parameter
                stream=False
            )
            # Extract the full text response
            text_content = response['choices'][0]['message']['content']
            return text_content

    except Exception as err:
        # Catch and print any errors
        print(f"An error occurred: {err}")
        return None


---

## Lesson

Oftentimes, you don't want to write full prompts, but instead want **prompt templates that can be modified later with additional input data before submitting to Llama**. This might come in handy if you want Llama to do the same thing every time, but the data that Llama uses for its task might be different each time.

Luckily, you can do this pretty easily by **separating the fixed skeleton of the prompt from variable user input, then substituting the user input into the prompt** before sending the full prompt to Llama.

Below, you'll walk step by step through how to write a substitutable prompt template, as well as how to substitute in user input.

### Examples

In this first example, you're asking Llama to act as an animal noise generator. Notice that the full prompt submitted to Llama is just the `PROMPT_TEMPLATE` substituted with the input (in this case, "Cow"). Notice that the word "Cow" replaces the `ANIMAL` placeholder via an f-string when you print out the full prompt.

**Note:** You don't have to call your placeholder variable anything in particular in practice. You called it `ANIMAL` in this example, but just as easily, you could have called it `CREATURE` or `A` (although it's generally good to have your variable names be specific and relevant so that your prompt template is easy to understand even without the substitution, just for user parseability). Just make sure that whatever you name your variable is what you use for the prompt template f-string.

In [ ]:
# Variable content
ANIMAL = "Cow"

# Prompt template with a placeholder for the variable content
PROMPT = f"I will tell you the name of an animal. Please respond with the noise that animal makes. {ANIMAL}"

# Print Llama's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Llama's response -------------------------------------")
print(get_completion(PROMPT))

Why would you want to separate and substitute inputs like this? Well, **prompt templates simplify repetitive tasks**. Let's say you build a prompt structure that invites third party users to submit content to the prompt (in this case the animal whose sound they want to generate). These third party users don't have to write or even see the full prompt. All they have to do is fill in variables.

You do this substitution here using variables and f-strings, but you can also do it with the format() method.

**Note:** Prompt templates can have as many variables as desired!

When introducing substitution variables like this, it is very important to **make sure Llama knows where variables start and end** (vs. instructions or task descriptions). Let's look at an example where there is no separation between the instructions and the substitution variable.

To our human eyes, it is very clear where the variable begins and ends in the prompt template below. However, in the fully substituted prompt, that delineation becomes unclear.

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Llama. {EMAIL} <----- Make this email more polite but don't change anything else about it."

# Print Llama's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Llama's response -------------------------------------")
print(get_completion(PROMPT))

Here, **Llama thinks "Yo Llama" is part of the email it's supposed to rewrite**! You can tell because it begins its rewrite with "Dear Llama". To the human eye, it's clear, particularly in the prompt template where the email begins and ends, but it becomes much less clear in the prompt after substitution.

How do you solve this? **Wrap the input in XML tags**! We did this below, and as you can see, there's no more "Dear Llama" in the output.

[XML tags](https://docs.anthropic.com/Llama/docs/use-xml-tags) are angle-bracket tags like `<tag></tag>`. They come in pairs and consist of an opening tag, such as `<tag>`, and a closing tag marked by a `/`, such as `</tag>`. XML tags are used to wrap around content, like this: `<tag>content</tag>`.

**Note:** While Llama can recognize and work with a wide range of separators and delimeters, we recommend that you **use specifically XML tags as separators** for Llama, as Llama was trained specifically to recognize XML tags as a prompt organizing mechanism. Outside of function calling, **there are no special sauce XML tags that Llama has been trained on that you should use to maximally boost your performance**. Anthropic has purposefully made Llama very malleable and customizable this way.

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Llama. <email>{EMAIL}</email> <----- Make this email more polite but don't change anything else about it."

# Print Llama's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Llama's response -------------------------------------")
print(get_completion(PROMPT))

Let's see another example of how XML tags can help us.

In the following prompt, **Llama incorrectly interprets what part of the prompt is the instruction vs. the input**. It incorrectly considers `Each is about an animal, like rabbits` to be part of the list due to the formatting, when the user (the one filling out the `SENTENCES` variable) presumably did not want that.

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f"""Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal like rabbits.
{SENTENCES}"""

# Print Llama's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Llama's response -------------------------------------")
print(get_completion(PROMPT))

To fix this, you just need to **surround the user input sentences in XML tags**. This shows Llama where the input data begins and ends despite the misleading hyphen before `Each is about an animal, like rabbits.`

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f""" Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal like rabbits.
<sentences>
{SENTENCES}
</sentences>"""

# Print Llama's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Llama's response -------------------------------------")
print(get_completion(PROMPT))

**Note:** In the incorrect version of the "Each is about an animal" prompt, you had to include the hyphen to get Llama to respond incorrectly in the way you wanted to for this example. This is an important lesson about prompting: **small details matter**! It's always worth it to **scrub your prompts for typos and grammatical errors**. Llama is sensitive to patterns (in its early years, before finetuning, it was a raw text-prediction tool), and it's more likely to make mistakes when you make mistakes, smarter when you sound smart, sillier when you sound silly, and so on.

If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## Exercises
- [Exercise 4.1 - Haiku Topic](#exercise-41---haiku-topic)
- [Exercise 4.2 - Dog Question with Typos](#exercise-42---dog-question-with-typos)
- [Exercise 4.3 - Dog Question Part 2](#exercise-42---dog-question-part-2)

### Exercise 4.1 - Haiku Topic
Modify the `PROMPT` so that it's a template that will take in a variable called `TOPIC` and output a haiku about the topic. This exercise is just meant to test your understanding of the variable templating structure with f-strings.

In [ ]:
# Variable content
TOPIC = "Pigs"

# Prompt template with a placeholder for the variable content
PROMPT = f"Create a haiku about "

# Get Llama's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("pigs", text.lower()) and re.search("haiku", text.lower()))

# Print Llama's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Llama's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ If you want a hint, run the cell below!

In [ ]:
print(hints.exercise_4_1_hint)

### Exercise 4.2 - Dog Question with Typos
Fix the `PROMPT` by adding XML tags so that Llama produces the right answer.

Try not to change anything else about the prompt. The messy and mistake-ridden writing is intentional, so you can see how Llama reacts to such mistakes.

In [ ]:
# Variable content
QUESTION = "are dogs brown"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hia its me i have a q jkaerjv dasasdv  lololirir {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Get Llama's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Print Llama's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Llama's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ If you want a hint, run the cell below!

In [ ]:
print(hints.exercise_4_2_hint)

### Exercise 4.3 - Dog Question Part 2
Fix the `PROMPT` **WITHOUT** adding XML tags. Instead, remove only one or two words from the prompt.

Just as with the above exercises, try not to change anything else about the prompt. This will show you what kind of language Llama can parse and understand.

In [ ]:
# Variable content
QUESTION = "are dogs brown"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hia its me i have a q jkaerjv dasasdv  lololirir {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Get Llama's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Print Llama's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Llama's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ If you want a hint, run the cell below!

In [ ]:
print(hints.exercise_4_3_hint)

### Congrats!

If you've solved all exercises up until this point, you're ready to move to the next chapter. Happy prompting!

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Llama's responses.

In [ ]:
# Variable content
ANIMAL = "Cow"

# Prompt template with a placeholder for the variable content
PROMPT = f"I will tell you the name of an animal. Please respond with the noise that animal makes. {ANIMAL}"

# Print Llama's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Llama's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Llama. {EMAIL} <----- Make this email more polite but don't change anything else about it."

# Print Llama's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Llama's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Llama. <email>{EMAIL}</email> <----- Make this email more polite but don't change anything else about it."

# Print Llama's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Llama's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f"""Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
{SENTENCES}"""

# Print Llama's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Llama's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f""" Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
<sentences>
{SENTENCES}
</sentences>"""

# Print Llama's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Llama's response -------------------------------------")
print(get_completion(PROMPT))